<a href="https://colab.research.google.com/github/varshachawan/SparkML/blob/master/DecisonTree_WineData_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Check the SQLContext
The entry point into all functionality in Spark SQL <br />
The SQLContext can be used to create a Spark Dataframe (as opposed to an RDD) from a data source

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirror.olnevhost.net/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [2]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-2.4.5-bin-hadoop2.7"
!echo $JAVA_HOME
import findspark
findspark.init()

/usr/lib/jvm/java-8-openjdk-amd64


<b>Dataset location: </b>https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data <br />
The same dataset we have used so far

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('Predicting the grape variety from wine characteristics') \
    .getOrCreate()

rawData = spark.read\
            .format('csv')\
            .option('header', 'false')\
            .load('./wine.data')

#### View the schema of the loaded DataFrame
* There are no column names
* All values are loaded as strings

In [0]:
rawData

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string]

#### View the values in the top 5 rows

In [0]:
rawData.show(5)

+---+-----+----+----+----+---+----+----+---+----+----+----+----+----+
|_c0|  _c1| _c2| _c3| _c4|_c5| _c6| _c7|_c8| _c9|_c10|_c11|_c12|_c13|
+---+-----+----+----+----+---+----+----+---+----+----+----+----+----+
|  1|14.23|1.71|2.43|15.6|127| 2.8|3.06|.28|2.29|5.64|1.04|3.92|1065|
|  1| 13.2|1.78|2.14|11.2|100|2.65|2.76|.26|1.28|4.38|1.05| 3.4|1050|
|  1|13.16|2.36|2.67|18.6|101| 2.8|3.24| .3|2.81|5.68|1.03|3.17|1185|
|  1|14.37|1.95| 2.5|16.8|113|3.85|3.49|.24|2.18| 7.8| .86|3.45|1480|
|  1|13.24|2.59|2.87|  21|118| 2.8|2.69|.39|1.82|4.32|1.04|2.93| 735|
+---+-----+----+----+----+---+----+----+---+----+----+----+----+----+
only showing top 5 rows



#### Assign names to each of the columns
And create a new dataframe from it

In [0]:
dataset = rawData.toDF('Label',
                'Alcohol',
                'MalicAcid',
                'Ash',
                'AshAlkalinity',
                'Magnesium',
                'TotalPhenols',
                'Flavanoids',
                'NonflavanoidPhenols',
                'Proanthocyanins',
                'ColorIntensity',
                'Hue',
                'OD',
                'Proline'
                )

#### Confirm that the dataset contains the column names

In [0]:
dataset

DataFrame[Label: string, Alcohol: string, MalicAcid: string, Ash: string, AshAlkalinity: string, Magnesium: string, TotalPhenols: string, Flavanoids: string, NonflavanoidPhenols: string, Proanthocyanins: string, ColorIntensity: string, Hue: string, OD: string, Proline: string]

#### View the dataset with the values

In [5]:
dataset.show(5)

+-----+-------+---------+----+-------------+---------+------------+----------+-------------------+---------------+--------------+----+----+-------+
|Label|Alcohol|MalicAcid| Ash|AshAlkalinity|Magnesium|TotalPhenols|Flavanoids|NonflavanoidPhenols|Proanthocyanins|ColorIntensity| Hue|  OD|Proline|
+-----+-------+---------+----+-------------+---------+------------+----------+-------------------+---------------+--------------+----+----+-------+
|    1|  14.23|     1.71|2.43|         15.6|      127|         2.8|      3.06|                .28|           2.29|          5.64|1.04|3.92|   1065|
|    1|   13.2|     1.78|2.14|         11.2|      100|        2.65|      2.76|                .26|           1.28|          4.38|1.05| 3.4|   1050|
|    1|  13.16|     2.36|2.67|         18.6|      101|         2.8|      3.24|                 .3|           2.81|          5.68|1.03|3.17|   1185|
|    1|  14.37|     1.95| 2.5|         16.8|      113|        3.85|      3.49|                .24|           2.1

#### Define a vectorize function to store the data in the required format for our ML models
The ML package needs data be put in a (label: Double, features: Vector) DataFrame format with correspondingly named fields. The vectorize() function does just that
* We perform a manual transformation of our dataset here
* Spark ML also supplies built-in transformers which we will use shortly

In [0]:
from pyspark.ml.linalg import Vectors

def vectorize(data):
    return data.rdd.map(lambda r: [r[0], Vectors.dense(r[1:])]).toDF(['label','features'])

#### Convert our data set into the vectorized format

In [0]:
vectorizedData = vectorize(dataset)

In [0]:
vectorizedData.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|[14.23,1.71,2.43,...|
|    1|[13.2,1.78,2.14,1...|
|    1|[13.16,2.36,2.67,...|
|    1|[14.37,1.95,2.5,1...|
|    1|[13.24,2.59,2.87,...|
+-----+--------------------+
only showing top 5 rows



#### View the transformed dataset
The features are now a DenseVector with an array of feature values

In [8]:
vectorizedData.take(5)

[Row(label='1', features=DenseVector([14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0])),
 Row(label='1', features=DenseVector([13.2, 1.78, 2.14, 11.2, 100.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.4, 1050.0])),
 Row(label='1', features=DenseVector([13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.68, 1.03, 3.17, 1185.0])),
 Row(label='1', features=DenseVector([14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0])),
 Row(label='1', features=DenseVector([13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]))]

#### StringIndexer 
* It's a feature transformer (can also be used for labels)
* Encodes a string column to a column of indices. The indices are in [0, numLabels), ordered by value frequencies, so the most frequent value gets index 0
* The label needs to be of type Double which will be handled by StringIndexer

In [0]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol='label',
                             outputCol='indexedLabel')

#### Transform the label in the vectorized dataset with the StringIndexer
We get a new label field called indexedLabel

In [10]:
indexedData = labelIndexer.fit(vectorizedData).transform(vectorizedData)
indexedData.take(10)

[Row(label='1', features=DenseVector([14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([13.2, 1.78, 2.14, 11.2, 100.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.4, 1050.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.68, 1.03, 3.17, 1185.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0]), indexedLabel=1.0),
 Row(label='1', features=DenseVector([14.39, 1.87, 2.45, 14.6, 96.0, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290.0]), indexedLabel=1.0),
 Row(label='1', features=D

#### Confirm that the indexedLabel is in Double format

In [0]:
indexedData

DataFrame[label: string, features: vector, indexedLabel: double]

In [0]:
indexedData.select('label').distinct().show()

+-----+
|label|
+-----+
|    3|
|    1|
|    2|
+-----+



In [0]:
indexedData.select('indexedLabel').distinct().show()

+------------+
|indexedLabel|
+------------+
|         0.0|
|         1.0|
|         2.0|
+------------+



#### Split the vectorized data into training and test sets

In [0]:
(trainingData, testData) = indexedData.randomSplit([0.8, 0.2])

### DecisionTree Classifier
* Specify the features and label columns
* <b>maxDepth: </b>The maximum depth of the decision tree
* <b>impurity: </b>We use gini instead of entropy. Gini measurement is the probability of a random sample being classified correctly. Entropy is a measure of information (seek to maximize information gain when making a split). Outputs generally don't vary much when either option is chosen, but entropy may take longer to compute as it calculates a logarithm

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

dtree = DecisionTreeClassifier(
    labelCol='indexedLabel', 
    featuresCol='features',
    maxDepth=3,
    impurity='gini'
)

#### Traing the model using the training data

In [0]:
model = dtree.fit(trainingData)

#### Use Spark ML's MulticlassClassificationEvaluator to evaluate the model
* Used to evaluate classification models
* It takes a set of labels and predictions as input
* Similar to (but not the same as MulticlassMetrics in MLLib)
* <b>metricName: </b>Can be precision, recall, weightedPrecision, weightedRecall and f1

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='indexedLabel',
                                              predictionCol='prediction', 
                                              metricName='f1',
                                              )

#### Transform the test data using our model to include predictions

In [16]:
transformed_data = model.transform(testData)
transformed_data.show(5)

+-----+--------------------+------------+--------------+--------------------+----------+
|label|            features|indexedLabel| rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------+--------------------+----------+
|    1|[13.39,1.77,2.62,...|         1.0|[0.0,44.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|    1|[13.56,1.73,2.46,...|         1.0|[0.0,44.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|    1|[13.68,1.83,2.36,...|         1.0|[51.0,2.0,0.0]|[0.96226415094339...|       0.0|
|    1|[13.77,1.9,2.68,1...|         1.0|[0.0,44.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|    1|[13.82,1.75,2.42,...|         1.0|[0.0,44.0,0.0]|       [0.0,1.0,0.0]|       1.0|
+-----+--------------------+------------+--------------+--------------------+----------+
only showing top 5 rows



#### Measure accuracy of model on the test data

In [21]:
print(evaluator.getMetricName(), 
      'accuracy:', 
      evaluator.evaluate(transformed_data))

f1 accuracy: 0.9641496598639456


#### View only the columns relevant for the predictions

In [0]:
predictions = transformed_data.select('indexedLabel', 'prediction', 'probability')
predictions.show(5)

+------------+----------+--------------------+
|indexedLabel|prediction|         probability|
+------------+----------+--------------------+
|         1.0|       1.0|       [0.0,1.0,0.0]|
|         1.0|       1.0|       [0.0,1.0,0.0]|
|         1.0|       1.0|       [0.0,1.0,0.0]|
|         1.0|       0.0|[0.97872340425531...|
|         1.0|       1.0|       [0.0,1.0,0.0]|
+------------+----------+--------------------+
only showing top 5 rows



#### Spark dataframes can also be converted to Pandas dataframes
View our predictions as a Pandas dataframe

In [0]:
predictions.toPandas().head()

,indexedLabel,prediction,probability
0,1.0,1.0,"[0.0, 1.0, 0.0]"
1,1.0,1.0,"[0.0, 1.0, 0.0]"
2,1.0,1.0,"[0.0, 1.0, 0.0]"
3,1.0,0.0,"[0.9787234042553191, 0.02127659574468085, 0.0]"
4,1.0,1.0,"[0.0, 1.0, 0.0]"
